In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
import cv2

# %matplotlib inline
%matplotlib qt

%config InlineBackend.figure_format = 'retina'

In [2]:
def load_image():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.jpg'
    return cv2.imread(path)

In [3]:
def mat_to_np():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.mat'
    mat_data = loadmat(path)
    corners = np.array(mat_data['corners'])
    K = np.array(mat_data['K'])
    v = np.array(mat_data['v'])
    return corners, K ,v

In [4]:
def to_cart(x):
    return x/x[-1]

In [5]:
def normalize(v):
    return v/(v @ v)**0.5

In [6]:
def compute_camera_center(P):
    M = P[:,:3]
    P4 = P[:,-1]
    C = -1*(np.linalg.inv(M) @ P4)
    return C

In [7]:
def compute_principal_axis(P):
    M = P[:,:3]
    m3 = P[-1,:3]
    v = np.linalg.det(M) * m3
    return normalize(v)

In [8]:
def camera_center_and_axis(P):
    C = compute_camera_center(P)
    p_axis = compute_principal_axis(P) 
    return C, p_axis

In [28]:
def calibrate(p, K):
    return np.linalg.inv(K) @ p

In [ ]:
def compute_3D_point(pi, x2D):
    s = -pi @ x2D
    x3D = x2D / s
    return x3D

Before calibration: origin of the image is in the upper left corner.
After calibration: origin of the image is a bit upper left of the center of the image.

In [64]:
def plot_2D(corners, img, plt_img=False):
    x = corners[0,:]
    y = corners[1,:]

    plt.figure(figsize=(10,8))
    plt.plot(x, y, 'o', color='red', label='Corners')
    plt.axis('equal')

    if plt_img is True:
        plt.imshow(img)
    else:
        plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.show()

In [49]:
def plot_3d(P1, s1, U):
    C1, p_axis1 = camera_center_and_axis(P1)
    # C2, p_axis2 = camera_center_and_axis(P2)

    plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')
    ax.plot(U[0], U[1], U[2], '.', ms=0.2, color='magenta', label='U')
    
    ax.plot(C1[0], C1[1], C1[2], 'o', color='red', label='Camera center 1')
    # ax.plot(C2[0], C2[1], C2[2], 'o', color='lime', label='Camera center 2')

    x_p_axis1 = C1[0] + s1*p_axis1[0]
    y_p_axis1 = C1[1] + s1*p_axis1[1]
    z_p_axis1 = C1[2] + s1*p_axis1[2]

    # x_p_axis2 = C2[0] + s2*p_axis2[0]
    # y_p_axis2 = C2[1] + s2*p_axis2[1]
    # z_p_axis2 = C2[2] + s2*p_axis2[2]

    ax.plot(x_p_axis1, y_p_axis1, z_p_axis1, 'o', color='blue', label='$s_1\\cdot$Principal axis 1')
    # ax.plot(x_p_axis2, y_p_axis2, z_p_axis2, 'o', color='cyan', label='$s_2\\cdot$Principal axis 2')

    ax.plot([x_p_axis1,C1[0]], [y_p_axis1,C1[1]], [z_p_axis1,C1[2]], '-', color='black')
    # ax.plot([x_p_axis2,C2[0]], [y_p_axis2,C2[1]], [z_p_axis2,C2[2]], '-', color='black')

    plt.legend(loc="lower right")
    plt.show()
    

In [65]:
img = load_image()
corners, K, v = mat_to_np()
corners_cal = calibrate(corners, K)

plot_2D(corners, img, plt_img=True)
plot_2D(corners_cal, img)

# plot_corners_cal(img, corners_cal)

corners_3D = [compute_3D_point(to_cart(v), p) for p in corners_cal]
P = np.concatenate((np.identity(3), np.zeros([3,1])), 1)


NameError: name 'compute_3D_point' is not defined

In [9]:
img = load_image()
corners, K, v = mat_to_np()
plot(img, corners)
C1, p_axis1 = camera_center_and_axis(P1)
C2, p_axis2 = camera_center_and_axis(P2)

print(P1 @ np.append(C1,1))
print(P2 @ np.append(C2,1), '\n')

print('Camera center 1:', C1, '\nPrincipal axis 1:', p_axis1)
print('\nCamera center 2:', C2, '\nPrincipal axis 2:', p_axis2)

s1 = 22
s2 = 18
plot_camera(P1, P2, s1, s2, to_cart(U))

NameError: name 'P1' is not defined